<a href="https://colab.research.google.com/github/Verose/ML_Applications_TAU/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import *
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, zero_one_loss
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import math
import os
import gzip

# Section 1: Weighted Random Forest implementation

In the first part, you are requested to implement a variation of Random Forest, which we will call "weighted" random forest (WRF vs. RF).


TODO 1: As a warm up exercise - 

We said that the bag size (for calculating out of bag error) of a tree in the RF ensemble is about 1/3 (to be more accurate it's 0.36) of the dataset. Please explain athematically how we got to this number (no need to show a formal proof, an explenation is good enough).

**Explanation:**

We wish to calculate the probability that a specific data item *x* is not sampled a tree.

We know that every tree samples *n* data items with replacement.

The probability that a tree does not sample *x* at a specific sample is (n-1)/n = 1-(1/n)

Meaning that the probability of not sampling *x* all is across *n* samples is (1-(1/n))^n

Take *n* to infinity get that (1-(1/n))^n -> e^(-1)~0.36

Meaning that the bag size of a tree is about 0.36~1/3


In [0]:
# TODO 2: implement WRF following the provided class API. You should support both, classification as
# well as regression (the type argument can be either "cat" or "reg"). You should use DecisionTreeClassifier
# and DecisionTreeRegressor as the underlying trees.

class WRF(BaseEstimator):
    def __init__(self, n_trees=100, max_depth=5, n_features=None, weight_type="div"):
        '''
          init a WRF classifier with the following parameters:

          n_trees: the number of trees to use.
          max_depth: the depth of each tree (will be passed along to DecisionTreeClassifier/DecisionTreeRegressor).
          n_features: the number of features to use for every split. The number should be given to DecisionTreeClassifier/Regressor as max_features.
          type: "cat" for categorization and "reg" for regression.
          weight_type: the tree weighting technique. 'div' for 1/error and 'sub' for 1-error.
        '''
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.weight_type = weight_type

    def fit(self, X, y):
        '''
          fit the classifier for the data X with response y.
        '''
        # <Your Code if needed>
        self.trees = []
        self.weights = []

        for _ in range(self.n_trees):
            tree = self.build_tree()
            X_tree, y_tree, X_oob, y_oob = self.bootstrap(X, y)
            tree.fit(X_tree, y_tree)
            weight = self.calculate_weight(tree, X_oob, y_oob)
            self.trees.append(tree)
            self.weights.append(weight)

        # Normalize the weights so they sum to 1
        # <Your code goes here>
        self.weights = self.softmax(self.weights)

        self.X_ = X
        self.y_ = y
        return self

    def softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def build_tree(self):
        pass

    def bootstrap(self, X, y):
        '''
          This method creates a bootstrap of the dataset (uniformly sample len(X) samples from X with repetitions).
          It returns X_tree, y_tree, X_oob, y_oob.
          X_tree, y_tree are the bootstrap collections for the given X and y.
          X_oob, y_oob are the out of bag remaining instances (the ones that were not sampled as part of the bootstrap)
        '''
        # <Your code goes here>
        if X.size == 0:
            return [], [], [], []

        length = X.shape[0]
        X_tree, Y_tree = [], []

        indexes = np.random.choice(range(length), length)
        for curr_index in indexes:
            X_tree.append(X[curr_index])
            Y_tree.append(y[curr_index])

        X_oob, Y_oob = [], []
        for curr_index in range(length):
            if curr_index not in indexes:
                X_oob.append(X[curr_index])
                Y_oob.append(y[curr_index])

        return X_tree, Y_tree, X_oob, Y_oob

    def calculate_weight(self, tree, X_oob, y_oob):
        '''
          This method calculates a weight for the given tree, based on it's performance on
          the OOB instances. We support two different types:
          if self.weight_type == 'div', we should return 1/error and if it's 'sub' we should
          return 1-error. The error is the normalized error rate of the tree on OOB instances.
          For classification use 0/1 loss error (i.e., count 1 for every wrong OOB instance and divide by the numbner of OOB instances),
          and for regression use mean square error of the OOB instances.
        '''

        # < Your code goes here>
        pass

    def predict(self, X):
        '''
          Predict the label/value of the given instances in the X matrix.
          For classification you should implement weighted voting, and for regression implement weighted mean.
          Return a list of predictions for the given instances.
        '''

        # <Your code goes here>
        pass

    def get_params(self, deep=True):
        return {
            "n_trees": self.n_trees,
            "max_depth": self.max_depth,
            "n_features": self.n_features,
            "weight_type": self.weight_type}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self


class WRFClassifier(WRF, ClassifierMixin):
    def build_tree(self):
        return DecisionTreeClassifier(max_depth=self.max_depth, max_features=self.n_features)

    def calculate_weight(self, tree, X_oob, y_oob):
        predictions = tree.predict(X_oob)
        error = zero_one_loss(y_oob, predictions)

        if self.weight_type == 'div':
            return 1 / (error + 0.0000001)
        else:
            return 1 - (error + 0.0000001)

    def predict(self, X):
        predictions_list = []
        for tree, weight in zip(self.trees, self.weights):
            if weight > 0:
                predictions = tree.predict(X)
                predictions_list.append((predictions, weight))
        y = []

        for ind, _ in enumerate(X):
            scores = {}

            for preds, weight in predictions_list:
                pred = preds[ind]

                if pred in scores:
                    scores[pred] += weight
                else:
                    scores[pred] = weight
            value = max(scores, key=scores.get)
            y += [value]
        return y


class WRFRegressor(WRF, RegressorMixin):
    def build_tree(self):
        return DecisionTreeRegressor(max_depth=self.max_depth, max_features=self.n_features)

    def calculate_weight(self, tree, X_oob, y_oob):
        predictions = tree.predict(X_oob)
        error = mean_squared_error(y_oob, predictions)

        if self.weight_type == 'div':
            return 1 / (error + 0.0000001)
        else:
            return 1 - (error + 0.0000001)

    def predict(self, X):
        predictions_list = []
        for tree, weight in zip(self.trees, self.weights):
            if weight > 0:
                predictions = tree.predict(X)
                predictions_list.append((predictions, weight))
        y = []

        for ind, _ in enumerate(X):
            mean = 0

            for preds, weight in predictions_list:
                mean += preds[ind] * weight
            y += [mean]
        return y


In [0]:
# prepare the datasets
!wget -P data/fashion http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget -P data/fashion http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget -P data/fashion http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget -P data/fashion http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
drive.mount('/content/drive')

--2018-12-12 08:46:12--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 52.219.74.7
Connecting to fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)|52.219.74.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [binary/octet-stream]
Saving to: ‘data/fashion/train-images-idx3-ubyte.gz.1’

train-images-idx3-u 100%[===================>]  25.20M  10.7MB/s    in 2.3s    

2018-12-12 08:46:15 (10.7 MB/s) - ‘data/fashion/train-images-idx3-ubyte.gz.1’ saved [26421880/26421880]

--2018-12-12 08:46:18--  http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
Resolving fashion-mnist.s3-website.eu-central-1.amazonaws.com (fashion-mnist.s3-website.eu-central-1.amazonaws.com)... 52.219.72.35
Connecting to fashion-mnist.s3-

# Section 2: Evaluation

In this section you are requested to evaluate your implementation, and compare it with RandomForestClassifier and RandomForestRegressor.

In [0]:
# TODO 3: Implement a tuning method for your classifier. 
# Note: you could potentially implement WRF as a sklearn classifier and then 
# simply use GridSearchCV inside. For those of you who wants to take this route, 
# you are welcome to modify the implementation of WRF accordingly. Check out: https://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator

def tune(classifier, X, y, scoring, arguments, cv=5):
    '''
      This method is doing exactly what GridSearchCV is doing for a sklearn classifier.
      It will run cross validation training with cv folds many times. Each time it will evaluate the CV "performance" on a different
      combination of the given arguments. You should check every combination of the given arguments and return a dictionary with
      the best argument combination. For classification, "performance" is accuracy. For Regression, "performance" is mean square error.

      classifier: it's the WRF classifier to tune
      X, y: the dataset to tune over
      arguments: a dictionary with keys are one of n_trees, max_depth, n_features, weight_type
      and the values are lists of values to test for each argument (see more in GridSearchCV)
    '''
  
    # <Your code goes here>
    clf = GridSearchCV(classifier, arguments, cv=cv, scoring=scoring)
    result = clf.fit(X, y)
    return result


In [0]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte.gz' % kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte.gz' % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels


def load_mnist_train_test():
    X_train, y_train = load_mnist('data/fashion', kind='train')
    X_test, y_test = load_mnist('data/fashion', kind='t10k')
    X_train = X_train[:7000]
    y_train = y_train[:7000]
    X_test = X_test[:5000]
    y_test = y_test[:5000]
    return X_train, X_test, y_train, y_test


def load_housing():
    data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML course/data/housing.csv')
    
    # data preparation
    ocean_prox = pd.get_dummies(data['ocean_proximity'], prefix='OCEAN')
    data = pd.concat([data, ocean_prox], axis=1)
    y = data["median_house_value"]
    X = data.drop(columns=["ocean_proximity", "median_house_value"])
    X = X.apply(lambda x: x.fillna(x.mean()))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
    return X_train.values, X_test.values, y_train.values, y_test.values


def tune_predict(X_test, X_train, y_test, y_train, rf, wrf):
    rf_parameters = {
        'n_estimators': (50, 100, 120),
        'max_depth': (10, 20, 30),
        'max_features': (1, 'sqrt'),
    }
    wrf_parameters = {
        'n_trees': (50, 100, 120),
        'max_depth': (10, 20, 30),
        'n_features': (1, 'sqrt'),
        'weight_type': ('div', 'sub'),
    }
    tune_rf = tune(rf, X_train, y_train, 'accuracy', rf_parameters)
    tune_wrf = tune(wrf, X_train, y_train, 'accuracy', wrf_parameters)
    rf = tune_rf.best_estimator_
    wrf = tune_wrf.best_estimator_
    rf_predict = rf.predict(X_test)
    wrf_predict = wrf.predict(X_test)
    return rf_predict, wrf_predict


In [0]:
# TODO 4: Evaluate your implementation and compare it RandomForestClassifier/Regressor provided by sklearn.

# For classification use the Fashion MNIST, but subsample the dataset to contain only 7K instances (out of the 60K available instances, you may simply select the first 7K instance from the data).
# - Tune both classifiers (WRF and RandomForestClassifier) before evaluation.
# - Evaluate both classifiers on the first 5K instances from the provided test data.
# - Report accuracy and a full confusion matrix for each classifier.

# For regression use the California housing dataset from Kaggle that we used in class:
# https://www.kaggle.com/harrywang/housing#housing.csv

# - Split the dataset to train and test (test_size=0.1, random_state=0)
# - Tune both regressors (WRF and RandomForestRegressor) before evaluation on the training set.
# - Evaluate both regressors on the test set.
# - Report mean square error.


In [0]:
def tune_predict(X_test, X_train, y_test, y_train, rf, wrf, scoring, rf_parameters, wrf_parameters):
    tune_rf = tune(rf, X_train, y_train, scoring, rf_parameters)
    tune_wrf = tune(wrf, X_train, y_train, scoring, wrf_parameters)
    print('GridSearch RF best params: {}'.format(tune_rf.best_params_))
    print('GridSearch WRF best params: {}'.format(tune_wrf.best_params_))
    rf = tune_rf.best_estimator_
    wrf = tune_wrf.best_estimator_
    rf_predict = rf.predict(X_test)
    wrf_predict = wrf.predict(X_test)
    return rf_predict, wrf_predict


In [0]:
# classifiers
X_train, X_test, y_train, y_test = load_mnist_train_test()
rf = RandomForestClassifier()
wrf = WRFClassifier()
rf_parameters = {
    'n_estimators': (100, 120),
    'max_depth': (10, 20, 25),
    'max_features': ('sqrt',),
}
wrf_parameters = {
    'n_trees': (100, 120),
    'max_depth': (10, 20, 25),
    'n_features': ('sqrt',),
    'weight_type': ('div', 'sub'),
}
rf_predict, wrf_predict = tune_predict(X_test, X_train, y_test, y_train, rf, wrf, 'accuracy', rf_parameters, wrf_parameters)
# Evaluation
print("Accuracy RandomForestClassifier", accuracy_score(y_test, rf_predict))
print("Confusion Matrix RandomForestClassifier", confusion_matrix(y_test, rf_predict))
print("Accuracy WeightedRandomForestClassifier", accuracy_score(y_test, wrf_predict))
print("Confusion Matrix WeightedRandomForestClassifier", confusion_matrix(y_test, wrf_predict))


GridSearch RF best params: {'max_depth': 25, 'max_features': 'sqrt', 'n_estimators': 120}
GridSearch WRF best params: {'max_depth': 20, 'n_features': 'sqrt', 'n_trees': 100, 'weight_type': 'sub'}
Accuracy RandomForestClassifier 0.8488
Confusion Matrix RandomForestClassifier [[404   2  14  25   2   1  53   0   6   0]
 [  1 459   3  17   1   0   0   0   0   0]
 [  5   0 403   5  75   0  32   0   1   0]
 [  8   4   5 444  18   0  20   0   1   0]
 [  0   2  73  26 393   1  25   0   1   0]
 [  0   0   0   0   0 457   0  21   0   7]
 [ 66   2  62  20  41   0 283   0   8   0]
 [  0   0   0   0   0  11   0 451   0  38]
 [  0   3   5   2   3   2   6   2 502   1]
 [  0   0   0   0   0   7   1  20   1 448]]
Accuracy WeightedRandomForestClassifier 0.8428
Confusion Matrix WeightedRandomForestClassifier [[407   2   8  23   3   2  56   0   6   0]
 [  0 460   3  16   1   0   1   0   0   0]
 [  6   0 397   5  76   0  35   0   2   0]
 [ 10   4   3 438  21   0  22   0   2   0]
 [  0   1  69  24 398   0  

In [0]:
# regressors
X_train, X_test, y_train, y_test = load_housing()
rf = RandomForestRegressor()
wrf = WRFRegressor()
rf_parameters = {
    'n_estimators': (5, 10),
    'max_depth': (3, 5, 8),
    'max_features': (1, 5, 'sqrt'),
}
wrf_parameters = {
    'n_trees': (5, 10),
    'max_depth': (3, 5, 8),
    'n_features': (1, 5, 'sqrt'),
    'weight_type': ('div', 'sub'),
}
rf_predict, wrf_predict = tune_predict(X_test, X_train, y_test, y_train, rf, wrf, 'neg_mean_squared_error', rf_parameters, wrf_parameters)
# Evaluation
print("Mean Squared Error RandomForestRegressor", mean_squared_error(y_test, rf_predict))
print("Mean Squared Error WeightedRandomForestRegressor", mean_squared_error(y_test, wrf_predict))


GridSearch RF best params: {'max_depth': 8, 'max_features': 5, 'n_estimators': 10}
GridSearch WRF best params: {'max_depth': 8, 'n_features': 5, 'n_trees': 10, 'weight_type': 'div'}
Mean Squared Error RandomForestRegressor 3316573275.093102
Mean Squared Error WeightedRandomForestRegressor 3223331713.7324686
